# utils安排
utils中间会包含logging \ scheduler \ optimizer \ evaluate之类的函数，就个人而言我在寻找函数分割的一个度。哪些函数可以当成独立的模块分离出来。

## WeChat_challenge
在微信挑战赛中他将setup_device \ setup_seed 单独分了出来，我其实一直有在想，将setup_device和setup_seed 分离出来是否多余，暂且记录一下
其实我觉得best_score的输出才是最重要的

## setup_device(args)

In [ ]:
import torch
import numpy as np
import random

def setup_device(args):
    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    args.n_gpu = torch.cuda.device_count()

## setup_seed

In [ ]:
def setup_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed

## evaluate

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def evaluate(predictions, labels):
    # prediction and labels are all level-2 class ids

    lv1_preds = [lv2id_to_lv1id(lv2id) for lv2id in predictions]
    lv1_labels = [lv2id_to_lv1id(lv2id) for lv2id in labels]

    lv2_f1_micro = f1_score(labels, predictions, average='micro')
    lv2_f1_macro = f1_score(labels, predictions, average='macro')
    lv1_f1_micro = f1_score(lv1_labels, lv1_preds, average='micro')
    lv1_f1_macro = f1_score(lv1_labels, lv1_preds, average='macro')
    mean_f1 = (lv2_f1_macro + lv1_f1_macro + lv1_f1_micro + lv2_f1_micro) / 4.0

    eval_results = {'lv1_acc': accuracy_score(lv1_labels, lv1_preds),
                    'lv2_acc': accuracy_score(labels, predictions),
                    'lv1_f1_micro': lv1_f1_micro,
                    'lv1_f1_macro': lv1_f1_macro,
                    'lv2_f1_micro': lv2_f1_micro,
                    'lv2_f1_macro': lv2_f1_macro,
                    'mean_f1': mean_f1}

    return eval_results

## unifiedprasing
这篇是非常早的semantic segmentation的文章，提出UperNet网络结构。在utils中他包含了一定的图片上色模块

## AverageMeter(object)
这里编写了一个计算总和的函数，但是我不太理解这个函数在项目中的用处.
👉self将class定义中的变量和函数变成实例变量和实例函数，作为类的成员，使得成员之间就能相互调用，不需要从外部调用数据和方法。实现了一个数据的封装。

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.initialized = False
        self.val = None
        self.avg = None
        self.sum = None
        self.count = None

    def initialize(self, val, weight):
        self.val = val
        self.avg = val
        self.sum = val * weight
        self.count = weight
        self.initialized = True

    def update(self, val, weight=1):
        if not self.initialized:
            self.initialize(val, weight)
        else:
            self.add(val, weight)

    def add(self, val, weight):
        self.val = val
        self.sum += val * weight
        self.count += weight
        self.avg = self.sum / self.count

    def value(self):
        return self.val

    def average(self):
        return self.avg

## accuracy
一个非常普遍的accuarcy的衡量方法。具体的acc scores文章有一个细致分类。

In [ ]:
def accuracy(preds, label):
    valid = (label >= 0)
    acc_sum = (valid * (preds == label)).sum()
    valid_sum = valid.sum()
    acc = float(acc_sum) / (valid_sum + 1e-10)
    return acc, valid_sum

## unique(ar, return_index=False, return_inverse = False)
具体还没有厘清楚拿来干什么的
np.asanyarray: 将非numpy存储的数据输入转化为numpy数据存储形式
flatten(): return a copy of the array collapsed into one dimension (返回一维数据)
np.empty(shape, dtype): return a new array of given shape and type
np.argsort(a, axis): return the indices that would sort an array

In [ ]:
def unique(ar, return_index=False, return_inverse=False, return_counts=False):
    ar = np.asanyarray(ar).flatten()

    optional_indices = return_index or return_inverse
    optional_returns = optional_indices or return_counts

    if ar.size ==0:
        if not optional_returns:
            # not return, ret = 0
            ret = ar
        else:
            ret = (ar,)
            if return_index:
                # return ret, size=0, dtype=bool
                ret += (np.empty(0, np.bool),)
            if return_inverse:
                ret += (np.empty(0, np.bool),)
            if return_counts:
                ret += (np.empty(0, np.intp),)
        return ret
    if optional_indices:
        perm = ar.argsort(kind='mergesort' if return_index else 'quicksort')
        aux = ar[perm]
    else:
        ar.sort()
        aux = ar